In [1]:
import tensorflow as tf
import tensorflow.contrib.seq2seq as seq2seq
from tensorflow.contrib.rnn import LSTMCell, LSTMStateTuple, GRUCell, MultiRNNCell
from tensorflow.contrib.rnn.python.ops.rnn_cell import _linear

from lib import data_utils, model_utils
from configs import model_config

import tensorflow as tf
import time
import math
import os
import sys

import numpy


In [2]:
#def __init__(self, config, use_lstm=True, forward_only=False,  attention=False ,sess=None):

config = model_config.Config()
use_lstm=True 
forward_only=True
attention=True

#config에서 모델의 설정값을 가져옵니다.
vocab_size = config.vocab_size
enc_hidden_size = config.enc_hidden_size
enc_num_layers = config.enc_num_layers
dec_hidden_size = config.dec_hidden_size
dec_num_layers = config.dec_num_layers
batch_size = config.batch_size
attention = attention


#학습 과정에서 가변 learning_rate 를 적용하기 위해서 tf.Variable 타입으로 선언합니다
learning_rate = tf.Variable(float(config.learning_rate), trainable=False)
#아래 op는 learning step 에서 두번이상 loss 가 증가헀을경우 learning_rate_decay_factor 를 곱해서
#기본값인 learning_rate_decay_factor = 0.99의 경우 본 op가 한번 실행될때마다 learning_rate 가 1% 감소시키는 효과를 가저옵니다
learning_rate_decay_op = learning_rate.assign(
    learning_rate * config.learning_rate_decay_factor)
#global_step 역시 학습 과정에서 계속 증가하여야 하는 값이므로 tf.Variable 형태로 선언하고 0으로 초기화합니다
global_step = tf.Variable(0, trainable=False)

max_gradient_norm = config.max_gradient_norm
buckets = config.buckets

#RNN의 내부 구조를 LSTM과 GRU중 선택으로 분기합니다.
if use_lstm:
    single_cell1 = LSTMCell(enc_hidden_size)
    single_cell2 = LSTMCell(dec_hidden_size)
else:
    single_cell1 = GRUCell(enc_hidden_size)
    single_cell2 = GRUCell(dec_hidden_size)

#multi layers 구조의 RNN 처리를 위한 부분 입니다. tensor 1.0 ~ 1.1 구간에서 해당 구현에 대한 이슈가 존재하여 본 예제에서는
#single layers의 구현을 가정하고 진행하겠습니다.
#seq2seq 모델의 경우 2 layers 부터 학습이 매우 어려워지는 경향을 보이며 4 layers 이상은 학습이 거의 되지 않는다 라는 경험적인 의견이 많습니다.
enc_cell = MultiRNNCell([single_cell1 for _ in range(enc_num_layers)])
dec_cell = MultiRNNCell([single_cell2 for _ in range(dec_num_layers)])

#인코딩 셀과 디코딩 셀을 self로 클래스 변수로 잡아 클래스내 다른 함수에서 이를 사용할것 입니다.
encoder_cell = enc_cell
decoder_cell = dec_cell


In [3]:
#def _init_data(self):
# encoder_inputs의 placeholder로 들어오는 2차원 matrix 예시 입니다.
#[
# [36, 6, 36, 6, 14, 5, 13, 35, 739, 41, 24, 103, EOS_ID],
# [3, 5, 13, 956, 3, 227, EOS_ID, GO_ID, 142, 331, 4, 17, 8, 112, 6, 155, 3, EOS_ID] , ...
#]

encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name="encoder_inputs")
encoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name="encoder_inputs_length")

# decoder_inputs의 placeholder로 들어오는 2차원 matrix 예시 입니다.
#[
# [GO_ID, 5, 15, 33, 12, 2021, 3,2274,EOS_ID],
# [GO_ID, 142, 331, 4, 17, 8, 112, 6, 155, 3, EOS_ID] , ...
#]

decoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name="decoder_inputs")
decoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name="decoder_inputs_length")

#np.array 타입의 슬라이싱을 이용하여 decoder_inputs의 GO 심볼이 위치하는 부분을 제거합니다
decoder_targets = decoder_inputs[1:, :]

#input 과 output 데이터를 버킷에 사이즈에 맞춰서 잘라냅니다
#기본값은 	model.config = buckets = [(8, 15)]로 이 경우에는 input의 경우 순서열의 최대 길이가 8 output의 경우 최대길이가 15로 잘리게 됩니다
temp_encoder_inputs = encoder_inputs[:buckets[-1][0], :]
encoder_inputs2 = temp_encoder_inputs
temp_decoder_inputs = decoder_inputs[:buckets[-1][1], :]
decoder_inputs2 = temp_decoder_inputs

# cross entropy 계산을 위해서 decoder_input 패딩된 데이터와 실제 데이터를 골라내기 위한 mask 입니다.
# getbatch 에서 받은 데이터를 주입할 placeholder
target_weights = tf.placeholder(shape=(None, None), dtype=tf.float32, name="target_weights")


## data_init

seq2seq 모델에서 입력값과 출력값(예측값)을 담당하는 encoder와 decoder의 데이터 파이프라인을 정의합니다.
placeholder 라는 구조는 lazy 하게 모델을 구성하고 차후에 데이터가 입력될 구조를 정의 합니다.

![](http://i.imgur.com/Cc22Moi.png)




In [4]:
with tf.variable_scope("embedding") as scope:
    #vocab_size * hidden_size 만큼의 크기를 가진 embedding_matrix를 생성 합니다.    
    enc_embedding_matrix = tf.get_variable(
        name="enc_embedding_matrix",
        shape=[vocab_size, enc_hidden_size],
        initializer=tf.contrib.layers.xavier_initializer(),
        dtype=tf.float32)

    dec_embedding_matrix = tf.get_variable(
        name="dec_embedding_matrix",
        shape=[vocab_size, dec_hidden_size],
        initializer=tf.contrib.layers.xavier_initializer(),
        dtype=tf.float32)

    # 연속된 단어의 index값으로 표현된 입력값을 각 인덱스의 one-hot으로 표현하고 이어서
    # embedding_vector화 하는 과정을 embedding_lookup을 통해서 쉽게 처리할 수 있습니다.

    encoder_inputs_embedded = tf.nn.embedding_lookup(
        enc_embedding_matrix, encoder_inputs2)

    decoder_inputs_embedded = tf.nn.embedding_lookup(
        dec_embedding_matrix, decoder_inputs2)


## init_embeddings
one-hot으로 표현된 단어 index를 hidden size * hidden_size 사이즈 만큼의 embedding vector로 생성 합니다.

![](http://i.imgur.com/ZOj61li.png)

In [5]:
with tf.variable_scope("encoder") as scope:
    # encoder_cell을 가지고 dynamic_rnn Layer를 생성하고
    # embedding_lookup 통해서 만든 embedding_matrix를 연결합니다.
    (encoder_outputs, encoder_state) = tf.nn.dynamic_rnn(cell=encoder_cell,
                                                                inputs=encoder_inputs_embedded,
                                                                sequence_length=encoder_inputs_length,
                                                                time_major=True, dtype=tf.float32)

## init_encoder

encoder_cell을 가지고 dynamic_rnn을 통해서 Encoder 구조를 정의 합니다.

![](http://i.imgur.com/AhmowtE.png)

In [6]:
#def _init_decoder(self, forward_only):
with tf.variable_scope("decoder") as scope:
    def output_fn(outputs):
        return tf.contrib.layers.linear(outputs, vocab_size, scope=scope)

    # attention_states: size [batch_size, max_time, num_units]
    attention_states = tf.transpose(encoder_outputs, [1, 0, 2])

    #encoder_outputs 을 가지고 attention network에 필요한 값을 생성합니다.
    (attention_keys, attention_values, attention_score_fn, attention_construct_fn) = (
        seq2seq.prepare_attention(
            attention_states=attention_states,
            attention_option="bahdanau",
            num_units=dec_hidden_size))

    #prediction 전용 모델인경우

    #decoder 함수로 inference를 사용하고
    #아래의 train 과정보다 더 많은 인자를 입력 받는데
    #전처리 과정에서 진행한 embedding 과정의 역순을 일부 자동으로 처리하기 위함입니다.
    decoder_fn = seq2seq.attention_decoder_fn_inference(
        output_fn=output_fn,
        encoder_state=encoder_state,
        attention_keys=attention_keys,
        attention_values=attention_values,
        attention_score_fn=attention_score_fn,
        attention_construct_fn=attention_construct_fn,
        embeddings=dec_embedding_matrix,
        start_of_sequence_id=model_config.GO_ID,
        end_of_sequence_id=model_config.EOS_ID,
        maximum_length=buckets[-1][1],
        num_decoder_symbols=vocab_size,
    )
    # rnn_decoder Layer를 생성합니다
    # encoder를 지나서 계산된 c는 decoder_fn의 인자를 통해 decoder에 연결되고
    # decoder_outputs에서 logit 형태로 예측값을 출력합니다.
    (decoder_outputs, decoder_state, decoder_context_state) = (
        seq2seq.dynamic_rnn_decoder(
            cell=decoder_cell,
            decoder_fn=decoder_fn,
            time_major=True,
            scope=scope,
        ))

    decoder_logits = decoder_outputs

    #vocab 사이즈 만큼의 각각 단어의 확률값으로 표현된 리스트의 순서열을 얻고
    #argmax 연산을 통해서 최대값을 찾게 됩니다.
    decoder_prediction = tf.argmax(decoder_logits, axis=-1, name='decoder_prediction')

    #loss 계산을 위한 logit과 targets을 출력
    logits = tf.transpose(decoder_logits, [1, 0, 2])
    targets = tf.transpose(decoder_targets, [1, 0])

    if not forward_only:
        #train 모델인 경우 loss 값을 정의
        seq_loss = seq2seq.sequence_loss(logits=logits, targets=targets,weights=target_weights)

    #tensor sever를 정의하여 모델의 학습 과정에서 파라메터과 모델구조를 저장할 수 있도록 합니다.
    saver = tf.train.Saver(tf.global_variables())        


## init_decoder
decoder_cell을 가지고 dynamic_rnn Layer를 생성하고 encoder 에서 나온 encoder_state를 연결합니다.  
attention의 경우 encoder 단계에서의 각 단계의 state와 output에서 중간 단계의 값을 다시 decoder 구조에 연결합니다.  
![](http://i.imgur.com/QRdjS8z.png)  

In [7]:
def prediction(session, in_encoder_inputs, in_encoder_inputs_length):
    #placeholder 로 정의한 데이터 파이프 라인에
    #get_batch 등으로 모델 외부에서 주입할 데이터를 정의합니다.
    input_feed = {
        encoder_inputs: in_encoder_inputs,
        encoder_inputs_length: in_encoder_inputs_length,
    }
    #학습 모델이 아니라면 출력에 필요한 OP 만 묶어서 계산하고
    prediction = session.run(decoder_prediction, input_feed)
    return prediction

import re
_WORD_SPLIT = re.compile("([.,!?\"':;)(])")
_DIGIT_RE = re.compile(r"\d{3,}")
def basic_tokenizer(sentence):
  """Very basic tokenizer: split the sentence into a list of tokens."""
  words = []
  for space_separated_fragment in sentence.strip().split():
    words.extend(re.split(_WORD_SPLIT, space_separated_fragment))
  return [w.lower() for w in words if w]

In [8]:
import tensorflow as tf
import time
import math
import os
import sys

from lib import data_utils, model_utils
from configs import model_config
from tensorflow.python.platform import gfile


#새로운 tensorflow 세션을 생성합니다 이후 sess 라는 이름으로 호출합니다
with tf.Session() as sess:

    #체크 포인트 파일의 상태를 가져옵니다.
    ckpt = tf.train.get_checkpoint_state(config.model_dir)

    #이미 학습된 모델파일이 있다면 불러옵니다.
    if ckpt and gfile.Exists("%s.index" % ckpt.model_checkpoint_path):
        print("Reading model parameters from %s" % ckpt.model_checkpoint_path)
        saver.restore(sess, ckpt.model_checkpoint_path)

    #forward_only 가 True 일떄 모델을 예측 전용의 모델을 생성합니다. 배치 사이즈가 1로 변경되고 모델 그래프에 옵티마이저가 제거 됩니다.
    forward_only = True
    config.batch_size = 1

    #vacab파일의 경로를 정의합니다. 여러 모델의 테스트를 위해서 사전 크기를 파일명뒤에 명시적으로 표시합니다.
    #예를들어 8000 사이즈의 vovab 파일은 vocab8000.in 입니다.
    vocab_path = os.path.join(config.data_dir, 'vocab%d.in' % config.vocab_size)

    #사전 파일을 로드합니다
    #vovab.get(word) 형태로 해당 word에 맞는 사전 index 값을 반환합니다
    #vovab.get(word) 값이 None 일경우 _UNK word로 취급합니다
    #vocab_rev[index) 향태로 해당 index에 맞는 word를 반환 합니다.
    vocab, vocab_rev = data_utils.load_vocabulary(vocab_path)
    
    #다음과 같은 형태도 가능하지만 문장부호의 처리를 위해서 basic_tokenizer를 이용합니다.
    #s = "Yeah Now's not a good time".lower().split(" ")
       
    #I'll keep it in mind.
    #What the bell happened?
    #I lost them, that's what happened.
    #How did they get away?
    #They ran. As fast as they could. Caught a train.
    #Which one pulled the trigger?
    #The Indian. I was about 30 yards away.
        
    s = "I'll keep it in mind"
    s = basic_tokenizer(s)
        
    #잘라진 단어를 vocab.get() 을 이용해서 ids 값으로 변환합니다
    inputs = []

    for i in s:
        #잘라진 소문자 단어로 사전에 인덱스를 구합니다.
        index = vocab.get(i)
        #index 가 None 라면 희귀한 UNK 단어입니다
        if index == None:
            #ids에 UNK_ID를 추가합니다
            inputs.append([model_config.UNK_ID])
        #index가 존재하는 단어라면
        else:
            #ids 값에 index를 추가합니다
            inputs.append([index])

    dev_inputs_length = [len(inputs)]

    predicted = prediction(sess, inputs, dev_inputs_length)


    print("\nInput string : ")  
    #입력한 문장을 ids2string 하여 출력합니다.
    s = ""
    for input in inputs:
        s += (vocab_rev[input[0]]) + " "
    print (s)

    print("\nPrediction Results : ")    
    #입력한 문장에 대한 예측값(답변)을 ids2string 하여 출력합니다.
    s = ""
    for i in predicted:
        s += (vocab_rev[i[0]]) + " "
    print (s)



Reading model parameters from ./nn_models/model.ckpt-30500

Input string : 
i ' ll keep it in mind 

Prediction Results : 
no . it ' s okay . i need it . . . one . . 
